# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
print(len(city_data_df))

# Display sample data
city_data_df.head()

565


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kotel'nikovo,47.6317,43.1461,10.65,77,100,10.24,RU,1701454804
1,1,ilulissat,69.2167,-51.1000,2.01,48,20,7.20,GL,1701454804
2,2,bereznik,62.8551,42.7072,-9.57,98,100,0.95,RU,1701454805
3,3,anadyr,64.7500,177.4833,-10.72,100,75,4.00,RU,1701454805
4,4,vernon,50.2581,-119.2691,0.16,76,100,3.58,CA,1701454806


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

17


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,ilulissat,69.2167,-51.1000,2.01,48,20,7.20,GL,1701454804
16,16,grytviken,-54.2811,-36.5092,5.07,59,1,5.86,GS,1701454814
23,23,muynoq,43.7683,59.0214,7.34,39,70,3.72,UZ,1701454819
78,78,ushuaia,-54.8000,-68.3000,9.81,46,40,4.12,AR,1701454747
101,101,hondo,32.4500,130.2000,9.94,60,85,6.42,JP,1701454870
121,121,margate,51.3813,1.3862,3.72,67,53,1.34,GB,1701454894
135,135,puerto natales,-51.7236,-72.4875,9.25,46,20,12.86,CL,1701454903
141,141,lerwick,60.1545,-1.1494,2.00,66,65,5.96,GB,1701454906
204,204,hastings,50.8552,0.5729,2.18,69,79,5.54,GB,1701454765
226,226,gaspe,48.8334,-64.4819,5.95,60,20,5.14,CA,1701454958


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,ilulissat,GL,69.2167,-51.1000,48,
16,grytviken,GS,-54.2811,-36.5092,59,
23,muynoq,UZ,43.7683,59.0214,39,
78,ushuaia,AR,-54.8000,-68.3000,46,
101,hondo,JP,32.4500,130.2000,60,
121,margate,GB,51.3813,1.3862,67,
135,puerto natales,CL,-51.7236,-72.4875,46,
141,lerwick,GB,60.1545,-1.1494,66,
204,hastings,GB,50.8552,0.5729,69,
226,gaspe,CA,48.8334,-64.4819,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ilulissat - nearest hotel: Best Western Plus Hotel Ilulissat
grytviken - nearest hotel: No hotel found
muynoq - nearest hotel: Lala
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
hondo - nearest hotel: 亀泉荘
margate - nearest hotel: The Reading Rooms
puerto natales - nearest hotel: Dorotea Patagonia Hostel
lerwick - nearest hotel: Grand Hotel
hastings - nearest hotel: White Rock Hotel
gaspe - nearest hotel: Hotel Plante
quellon - nearest hotel: Aliwen
delijan - nearest hotel: هتل سبز کارون
konya - nearest hotel: No hotel found
hasaki - nearest hotel: 大新旅館
salobrena - nearest hotel: Hostal La Paloma
fasa - nearest hotel: No hotel found
ancud - nearest hotel: Cabañas


,City,Country,Lat,Lng,Humidity,Hotel Name
1,ilulissat,GL,69.2167,-51.1000,48,Best Western Plus Hotel Ilulissat
16,grytviken,GS,-54.2811,-36.5092,59,No hotel found
23,muynoq,UZ,43.7683,59.0214,39,Lala
78,ushuaia,AR,-54.8000,-68.3000,46,Apart Hotel Aires del Beagle
101,hondo,JP,32.4500,130.2000,60,亀泉荘
121,margate,GB,51.3813,1.3862,67,The Reading Rooms
135,puerto natales,CL,-51.7236,-72.4875,46,Dorotea Patagonia Hostel
141,lerwick,GB,60.1545,-1.1494,66,Grand Hotel
204,hastings,GB,50.8552,0.5729,69,White Rock Hotel
226,gaspe,CA,48.8334,-64.4819,60,Hotel Plante


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)